In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
# читаем таблицу

tab2 = pd.read_csv('../input/retropress-temporal-markup/temporal-thesaurus-analytical-corpora-test.csv')

# переводим все буквы в нижний регистр

tab2["Id"] = [i.lower() for i in tab2["Id"]]

# словарь для перевода буквенных чисел в записанные цифрами

dig = {"одинадцат":"11 ", "двенадцат":"12 ", "тринадцат":"13 ", "четырнадцат":"14 ", "пятнадцат":"15 ", "шестнадцат":"16 ", 
       "семнадцат":"17 ", "восемнадцат":"18 ", "девятнадцат":"19 ", "двадцат":"20 ", " один":"01 ", " одн":"01 ", "перв":"01 ", 
       "два ":"02 ", "второ":" 02", "три ":"03 ", "трет":"03 ", "четыр ":"04 ", "четверт":"04 ", "пят":"05 ",
       "шест":"06 ", "семь":"07 ", "седьм":"07 ", "восемь":"08 ", "восьм":"08 ", "девят":"9", "десят":"10", "ноль":" 0"}

# транслируем все числа, записанные буквами, в записанные цифрами

for r in range(len(tab2["Id"])):
    for i in dig.keys():
        if i in tab2["Id"][r]:
            tab2["Id"][r] = re.sub(i,dig[i],tab2["Id"][r])
            
# словарь для месяцев

d = {"янв":"01 ", "фев":"02 ", "мар":"03 ", "апр":"04 ", "май":"05 ", "мая":"05 ", "июн":"06 ", 
     "июл":"07 ", "авг":"08 ", "сент":"09 ", "окт":"10 ", "ноя":"11 ", "дек":"12 "}

# заменяем месяц на его числовой аналог

for r in range(len(tab2["Id"])):
    for i in d.keys():
        if i in tab2["Id"][r]:
            tab2["Id"][r] = re.sub(i,d[i],tab2["Id"][r])
            
# достаем день, месяц и год

tab2["day"] = [re.search("(\d{1,2}).+?(\d{1,2}).+?(\d{4})", i).group(1) 
               if re.search("(\d{1,2}).+?(\d{1,2}).+?(\d{4})", i) else "" for i in tab2["Id"]]
tab2["month"] = [re.search("(\d{1,2}).+?(\d{1,2}).+?(\d{4})", i).group(2) 
               if re.search("(\d{1,2}).+?(\d{1,2}).+?(\d{4})", i) else "" for i in tab2["Id"]]
tab2["year"] = [re.search("(\d{1,2}).+?(\d{1,2}).+?(\d{4})", i).group(3) 
               if re.search("(\d{1,2}).+?(\d{1,2}).+?(\d{4})", i) else "2020" for i in tab2["Id"]]

# собираем дату

tab2['Expected'] = ["" for i in range(len(tab2["Id"]))]
for i in range(len(tab2["Id"])):
    tab2['Expected'][i] = '{year}-{month}-{day}'.format(year=str(tab2["year"][i]), month=str(tab2["month"][i]), day=str(tab2["day"][i]))
    
# записываем в отдельный датасет и сохраняем

tab_final = tab2[['Id', 'Expected']]

tab_final.to_csv("submission.csv", index=False)